# 01 - Collecte de Données : Ligue 1

## Phase 0 — Validation du Modèle

**Objectif :** Récupérer les résultats de matchs terminés pour la Ligue 1 via l'API Football-Data.org.

Nous collectons deux saisons :
- **2023-24** → Données d'entraînement (training set)
- **2024-25** → Données de test walk-forward (out-of-sample)

Cette séparation temporelle est cruciale : on entraîne sur le passé et on valide sur le futur,
exactement comme on ferait en production.

### Sources de données
| Source | Usage | Quota |
|--------|-------|-------|
| Football-Data.org | Scores, résultats, journées | 100 req/jour (gratuit) |
| The Odds API | Cotes de bookmakers (optionnel) | 500 req/mois (gratuit) |

---
## 1. Configuration et imports

In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv

# --- Chemins du projet ---
# On remonte d'un niveau depuis notebooks/ pour atteindre la racine du projet
PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"

# Créer le dossier raw/ s'il n'existe pas
RAW_DIR.mkdir(parents=True, exist_ok=True)

# --- Ajouter src/ au path pour importer nos modules ---
SRC_DIR = str(PROJECT_ROOT / "src")
if SRC_DIR not in sys.path:
    sys.path.insert(0, SRC_DIR)

# --- Charger les variables d'environnement ---
load_dotenv(PROJECT_ROOT / ".env")

print(f"Racine du projet : {PROJECT_ROOT}")
print(f"Dossier données brutes : {RAW_DIR}")
print(f"Clé Football-Data.org : {'✓ configurée' if os.getenv('FOOTBALL_DATA_ORG_KEY') else '✗ manquante'}")
print(f"Clé Odds API : {'✓ configurée' if os.getenv('ODDS_API_KEY') else '✗ manquante (optionnel)'}")

In [ ]:
# Import du client Football-Data.org depuis src/data/
from data.football_data_org import FootballDataClient

# Initialisation du client
api_key = os.getenv("FOOTBALL_DATA_ORG_KEY")
if not api_key:
    raise ValueError(
        "La clé FOOTBALL_DATA_ORG_KEY est manquante.\n"
        "Créez un fichier .env à la racine du projet avec :\n"
        "FOOTBALL_DATA_ORG_KEY=votre_clé\n"
        "Inscription gratuite : https://www.football-data.org/"
    )

client = FootballDataClient(api_key=api_key)
print("Client Football-Data.org initialisé.")

---
## 2. Collecte des matchs Ligue 1 2023-24 (entraînement)

On récupère tous les matchs terminés (status=FINISHED) de la saison 2023-24.
Le paramètre `season=2023` correspond à la saison qui commence en 2023 (i.e. 2023-24).

In [ ]:
# Récupération des matchs terminés 2023-24
matches_2023 = client.get_finished_matches_with_scores(
    league="ligue_1",
    season=2023,
)

df_2023 = pd.DataFrame(matches_2023)
print(f"Matchs récupérés pour 2023-24 : {len(df_2023)}")
df_2023.head()

In [ ]:
# Nettoyage et typage
df_2023["kickoff"] = pd.to_datetime(df_2023["kickoff"])
df_2023["home_score"] = df_2023["home_score"].astype(int)
df_2023["away_score"] = df_2023["away_score"].astype(int)
df_2023["matchday"] = df_2023["matchday"].astype(int)

# Tri chronologique
df_2023 = df_2023.sort_values("kickoff").reset_index(drop=True)

# Colonnes dérivées utiles
df_2023["total_goals"] = df_2023["home_score"] + df_2023["away_score"]
df_2023["result"] = df_2023.apply(
    lambda r: "H" if r["home_score"] > r["away_score"]
    else ("A" if r["home_score"] < r["away_score"] else "D"),
    axis=1,
)

print(f"Période : {df_2023['kickoff'].min().date()} → {df_2023['kickoff'].max().date()}")
print(f"Journées : {df_2023['matchday'].min()} à {df_2023['matchday'].max()}")
df_2023.head(10)

---
## 3. Collecte des matchs Ligue 1 2024-25 (test walk-forward)

Même chose pour la saison en cours. On ne garde que les matchs déjà joués.

In [ ]:
# Récupération des matchs terminés 2024-25
matches_2024 = client.get_finished_matches_with_scores(
    league="ligue_1",
    season=2024,
)

df_2024 = pd.DataFrame(matches_2024)
print(f"Matchs récupérés pour 2024-25 : {len(df_2024)}")
df_2024.head()

In [ ]:
# Nettoyage et typage (même traitement que 2023-24)
df_2024["kickoff"] = pd.to_datetime(df_2024["kickoff"])
df_2024["home_score"] = df_2024["home_score"].astype(int)
df_2024["away_score"] = df_2024["away_score"].astype(int)
df_2024["matchday"] = df_2024["matchday"].astype(int)

df_2024 = df_2024.sort_values("kickoff").reset_index(drop=True)

df_2024["total_goals"] = df_2024["home_score"] + df_2024["away_score"]
df_2024["result"] = df_2024.apply(
    lambda r: "H" if r["home_score"] > r["away_score"]
    else ("A" if r["home_score"] < r["away_score"] else "D"),
    axis=1,
)

print(f"Période : {df_2024['kickoff'].min().date()} → {df_2024['kickoff'].max().date()}")
print(f"Journées : {df_2024['matchday'].min()} à {df_2024['matchday'].max()}")
df_2024.head(10)

In [ ]:
# Nombre de requêtes API utilisées jusqu'ici
print(f"Requêtes API utilisées : {client.requests_used} / 100 (quota journalier)")

---
## 4. Sauvegarde en CSV

On sauvegarde les données brutes dans `data/raw/` pour ne pas
avoir à refaire les appels API à chaque exécution.

In [ ]:
# Sauvegarde des CSV
path_2023 = RAW_DIR / "ligue1_2023.csv"
path_2024 = RAW_DIR / "ligue1_2024.csv"

df_2023.to_csv(path_2023, index=False)
df_2024.to_csv(path_2024, index=False)

print(f"Sauvegardé : {path_2023} ({len(df_2023)} lignes, {path_2023.stat().st_size / 1024:.1f} Ko)")
print(f"Sauvegardé : {path_2024} ({len(df_2024)} lignes, {path_2024.stat().st_size / 1024:.1f} Ko)")

---
## 5. Statistiques descriptives

Vérifions que les données sont cohérentes avec ce qu'on attend de la Ligue 1.

In [ ]:
def afficher_stats(df: pd.DataFrame, saison: str) -> None:
    """Affiche les statistiques clés d'une saison."""
    n = len(df)
    print(f"{'=' * 50}")
    print(f"  LIGUE 1 {saison}")
    print(f"{'=' * 50}")
    print(f"  Nombre de matchs       : {n}")
    print(f"  Équipes distinctes     : {df['home_team'].nunique()}")
    print()

    # Répartition des résultats
    counts = df["result"].value_counts()
    print("  Répartition des résultats :")
    for label, full_name in [("H", "Victoire domicile"), ("D", "Match nul"), ("A", "Victoire extérieur")]:
        c = counts.get(label, 0)
        pct = c / n * 100
        print(f"    {full_name:25s} : {c:3d}  ({pct:5.1f}%)")
    print()

    # Buts
    print("  Statistiques de buts :")
    print(f"    Moyenne buts/match    : {df['total_goals'].mean():.2f}")
    print(f"    Moyenne buts domicile : {df['home_score'].mean():.2f}")
    print(f"    Moyenne buts extérieur: {df['away_score'].mean():.2f}")
    print(f"    Matchs > 2.5 buts    : {(df['total_goals'] > 2.5).sum():3d}  ({(df['total_goals'] > 2.5).mean() * 100:.1f}%)")
    print(f"    Matchs 0-0           : {((df['home_score'] == 0) & (df['away_score'] == 0)).sum()}")
    print()

In [ ]:
afficher_stats(df_2023, "2023-24")
afficher_stats(df_2024, "2024-25")

In [ ]:
# Vérification rapide : une saison Ligue 1 complète = 306 matchs (18 x 17)
# Depuis 2023-24 la Ligue 1 a 18 équipes au lieu de 20
EXPECTED_FULL_SEASON = 306

if len(df_2023) == EXPECTED_FULL_SEASON:
    print(f"2023-24 : saison complète ({EXPECTED_FULL_SEASON} matchs) ✓")
else:
    print(f"2023-24 : {len(df_2023)} / {EXPECTED_FULL_SEASON} matchs")

if len(df_2024) == EXPECTED_FULL_SEASON:
    print(f"2024-25 : saison complète ({EXPECTED_FULL_SEASON} matchs) ✓")
else:
    print(f"2024-25 : {len(df_2024)} / {EXPECTED_FULL_SEASON} matchs (saison en cours)")

---
## 6. (Optionnel) Récupération des cotes — The Odds API

Si la clé `ODDS_API_KEY` est configurée, on récupère les cotes actuelles
pour les matchs à venir. Ces cotes serviront de benchmark : notre modèle
doit battre les cotes implicites des bookmakers pour avoir de la valeur.

**Note :** The Odds API ne fournit que les cotes des matchs à venir,
pas les cotes historiques. Pour les cotes historiques (nécessaires en
backtesting), il faudra utiliser une autre source ou les collecter au fil du temps.

In [ ]:
odds_api_key = os.getenv("ODDS_API_KEY")

if not odds_api_key:
    print("Clé ODDS_API_KEY non configurée — cette section est ignorée.")
    print("Pour récupérer les cotes, ajoutez ODDS_API_KEY dans votre .env")
    print("Inscription : https://the-odds-api.com/")
else:
    from data.odds_api import OddsAPIClient, extract_best_odds, remove_margin

    odds_client = OddsAPIClient(api_key=odds_api_key)

    # Récupérer les cotes pour les matchs à venir de Ligue 1
    try:
        odds_data = odds_client.get_odds(
            league="ligue_1",
            markets="h2h,totals",
            regions="eu",
        )

        if odds_data:
            print(f"Cotes récupérées pour {len(odds_data)} matchs à venir.")
            print(f"Requêtes restantes : {odds_client.remaining_requests}")
            print()

            # Afficher les meilleures cotes pour chaque match
            rows = []
            for match in odds_data:
                best = extract_best_odds(match)
                fair = remove_margin(
                    best["home"]["odds"],
                    best["draw"]["odds"],
                    best["away"]["odds"],
                )
                rows.append({
                    "home_team": match.get("home_team", ""),
                    "away_team": match.get("away_team", ""),
                    "home_odds": best["home"]["odds"],
                    "draw_odds": best["draw"]["odds"],
                    "away_odds": best["away"]["odds"],
                    "over25_odds": best["over25"]["odds"],
                    "under25_odds": best["under25"]["odds"],
                    "fair_home": round(fair["home"], 3),
                    "fair_draw": round(fair["draw"], 3),
                    "fair_away": round(fair["away"], 3),
                    "overround": round(fair["overround"], 3),
                })

            df_odds = pd.DataFrame(rows)

            # Sauvegarder les cotes
            odds_path = RAW_DIR / "ligue1_odds_current.csv"
            df_odds.to_csv(odds_path, index=False)
            print(f"Sauvegardé : {odds_path}")
            print()
            display(df_odds)
        else:
            print("Aucun match à venir trouvé (hors saison ou trêve).")

        odds_client.close()

    except Exception as e:
        print(f"Erreur lors de la récupération des cotes : {e}")
        odds_client.close()

---
## 7. Fermeture et résumé

On ferme proprement le client HTTP et on affiche un récapitulatif.

In [ ]:
# Fermer le client HTTP
client.close()

print("Résumé de la collecte")
print("=" * 40)
print(f"Fichiers générés :")
for f in sorted(RAW_DIR.glob("ligue1_*.csv")):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name:30s} ({size_kb:.1f} Ko)")
print()
print(f"Requêtes API consommées : {client.requests_used}")
print()
print("Prochaine étape : 02_feature_engineering.ipynb")